<center><h1><strong> Twitter Sentiment Analysis using ML </strong></h1>

### Importing Dependencies

In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import nltk
from sklearn.utils import resample

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kindn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Data Preprocessing

In [4]:
df = pd.read_csv(r"C:\Users\kindn\OneDrive\Documents\dataset.csv",encoding='ISO-8859-1')
print("Dataset have been read.")

Dataset have been read.


In [11]:
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [35]:
# checking the number of rows and columns df.shape

In [5]:
df.shape

(1048575, 6)

In [6]:
# naming the cols and reading the dataset again
col_names = ['target','id','date','flag','user','text']
df = pd.read_csv(r"C:\Users\kindn\OneDrive\Documents\dataset.csv", names=col_names, encoding='ISO-8859-1')
print("Dataset with column names have been read.")

Dataset with column names have been read.


In [7]:
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [8]:
df.shape

(1048576, 6)

In [9]:
# finding the missing values in the dataset
df.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

<code> There are no missing values in the dataset. </code>

In [10]:
# finding the distribution of "target" column
df['target'].value_counts()

target
0    800000
4    248576
Name: count, dtype: int64

For our own convenience we can change Label "4" into "1". 1 means positive

In [11]:
df.replace({'target':{4:1}}, inplace=True)
df['target'].value_counts()

target
0    800000
1    248576
Name: count, dtype: int64

<code> Now, <br> 0 --> Negative Tweet <br> 1 --> Positive Tweet </code>

In [12]:
df_negative=df[df['target'] == 0]
df_positive=df[df['target'] == 1]


In [13]:
df_positive_unsampled = resample(df_positive,replace=True,n_samples=len(df_negative),random_state=42)
df_balanced = pd.concat([df_negative,df_positive_unsampled])

In [14]:
df=df_balanced
df['target'].value_counts()


target
0    800000
1    800000
Name: count, dtype: int64

In [15]:
negative=df[df['target'] == 0]
positive=df[df['target'] == 1]

In [16]:
positive_sample=positive.sample(n=900,random_state=25)
negative_sample=negative.sample(n=900,random_state=25)

In [17]:
sampled_df=pd.concat([positive_sample,negative_sample])

In [18]:
sampled_df=sampled_df.sample(frac=1,random_state=1).reset_index(drop=True)
df=sampled_df


In [19]:
df['target'].value_counts()

target
0    900
1    900
Name: count, dtype: int64

In [20]:
df.head()

,target,id,date,flag,user,text
0,0,2222442595,Thu Jun 18 06:59:28 PDT 2009,NO_QUERY,vortechs2000,@kpreslan I would have if I had an iPhone....
1,1,1833367298,Sun May 17 23:04:04 PDT 2009,NO_QUERY,Eunmi_Jeong,Today is monday. The Seoul in Korea is......ve...
2,1,1835563314,Mon May 18 06:16:23 PDT 2009,NO_QUERY,theclarkster,@MrsClarkster Thank you xxx
3,0,1574005963,Tue Apr 21 03:51:38 PDT 2009,NO_QUERY,Marneezy,@FFEJyourself LOL. I love how its almost 4 a.m...
4,0,1968170692,Fri May 29 20:53:53 PDT 2009,NO_QUERY,PinkerJewel,@hollybird Aww.


### Steming

<code>STEMMING:</code> The process of reducing a word to it's root word.

e.g: diving, diver, dived == dive

In [21]:
port_stem = PorterStemmer()
port_stem

<PorterStemmer>

In [22]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    
    return stemmed_content

In [23]:
df['stemmed_content'] = df['text'].apply(stemming)  

In [24]:
df.head()

,target,id,date,flag,user,text,stemmed_content
0,0,2222442595,Thu Jun 18 06:59:28 PDT 2009,NO_QUERY,vortechs2000,@kpreslan I would have if I had an iPhone....,kpreslan would iphon
1,1,1833367298,Sun May 17 23:04:04 PDT 2009,NO_QUERY,Eunmi_Jeong,Today is monday. The Seoul in Korea is......ve...,today monday seoul korea sunni day happi go st...
2,1,1835563314,Mon May 18 06:16:23 PDT 2009,NO_QUERY,theclarkster,@MrsClarkster Thank you xxx,mrsclarkster thank xxx
3,0,1574005963,Tue Apr 21 03:51:38 PDT 2009,NO_QUERY,Marneezy,@FFEJyourself LOL. I love how its almost 4 a.m...,ffejyourself lol love almost time think went b...
4,0,1968170692,Fri May 29 20:53:53 PDT 2009,NO_QUERY,PinkerJewel,@hollybird Aww.,hollybird aww


In [25]:
print(df['stemmed_content'])


0                                    kpreslan would iphon
1       today monday seoul korea sunni day happi go st...
2                                  mrsclarkster thank xxx
3       ffejyourself lol love almost time think went b...
4                                           hollybird aww
                              ...                        
1795                                 sick wife bday doubl
1796    saw possibl gh final band list frig slipknot s...
1797                                                 cold
1798                                happi birthday sister
1799                                  final go sleep hour
Name: stemmed_content, Length: 1800, dtype: object


In [26]:
print(df['target'])

0       0
1       1
2       1
3       0
4       0
       ..
1795    0
1796    0
1797    0
1798    1
1799    0
Name: target, Length: 1800, dtype: int64


In [27]:
# seperating stemmed data and label
X = df['stemmed_content'].values
Y = df['target'].values

In [28]:
print(X)

['kpreslan would iphon'
 'today monday seoul korea sunni day happi go studi everyth nice day'
 'mrsclarkster thank xxx' ... 'cold' 'happi birthday sister'
 'final go sleep hour']


In [29]:
print(Y)

[0 1 1 ... 0 1 0]


### Splitting Data into Training and Testing Data

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, stratify=Y, random_state=2)

In [31]:
print(X.shape,X_train.shape,X_test.shape)

(1800,) (1440,) (360,)


In [32]:
print(X_train)

['djbeatbean tristeza ahshau' 'http twitpic com fatp poor car hit deer'
 'happi miss sleepi love' ...
 'oh ps sinc alreadi probabl say quot happi mother day quot mom mother'
 'monipython awh see twitter phone set get messag'
 'jordanknight realli hope check post new one sometim soon http bit ly b']


In [33]:
print(X_test)

['olegsmith go go go go go amaz friend go laugh good eat mr smith'
 'shaundiviney foot feel ur arm dw kiss betr u like doubt u wanna kiss foot hey ne thing possibl'
 'dannymcfli get feel like redhead guess there point tri amp far away'
 'dubaysin yeah bad nvr happen sigh' 'dannygokey feelin mutual danni boy'
 'ugh feel horribl sick' 'mszjoycii yeah cinema play screen n cant wait c'
 'thesongsiwrot thought due tomorrow know wrong'
 'terrytokyo wish could seen girl today well besid across crowd haah hope fun'
 'nicolejensen welcom' 'last day internet net whole week lol'
 'pool awesom two hour got home took shower went mcdonald stomach hurt'
 'booo work comput say perezhilton websit r rate im allow look sure littl peek wouldnt hurt'
 'celebr papa day work tomorrow suppos rain day'
 'thepistol http twitpic com cb alex love u hah bad ur come sf anymor wait til u guy come back'
 'booo ee suck daniella bring sam back happi'
 'christinecain run chri run good c u yest great day go bac holland a

As ML algorithms do not recognize textual data, so we have to convert them into numerical data.

In [34]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [35]:
print(X_train)

  (0, 66)	0.5773502691896257
  (0, 3430)	0.5773502691896257
  (0, 869)	0.5773502691896257
  (1, 803)	0.44495336269235797
  (1, 1476)	0.3635759511340483
  (1, 500)	0.3566619227458396
  (1, 2511)	0.3350759655265712
  (1, 1097)	0.44495336269235797
  (1, 635)	0.27058589266373456
  (1, 3467)	0.3069243571674121
  (1, 1533)	0.25483842267781276
  (2, 1966)	0.4090710550016978
  (2, 2975)	0.641974050025241
  (2, 2119)	0.44022152246794305
  (2, 1408)	0.47616719984685374
  (3, 776)	0.38537976711657007
  (3, 3432)	0.414922393388852
  (3, 1262)	0.414922393388852
  (3, 1267)	0.21155184278406447
  (3, 1947)	0.414922393388852
  (3, 3580)	0.28371740508554666
  (3, 2481)	0.32184361945235995
  (3, 544)	0.325833690668373
  (4, 1931)	0.49415919579988393
  (4, 387)	0.5220865582240161
  :	:
  (1437, 2945)	0.2577489781766171
  (1437, 2609)	0.4313696627346539
  (1437, 2322)	0.20699835370064468
  (1437, 781)	0.16256850378903628
  (1437, 1408)	0.21707431359415819
  (1438, 230)	0.4511096582961547
  (1438, 2159)	0.

In [36]:
print(X_test)

  (0, 2192)	0.2903582584070109
  (0, 1859)	0.26380648301332965
  (0, 1307)	0.1408888458197244
  (0, 1298)	0.82747269932311
  (0, 1207)	0.21317515221840921
  (0, 954)	0.21317515221840921
  (0, 109)	0.22505598247283545
  (1, 3587)	0.21563163507375394
  (1, 3523)	0.2455104317696265
  (1, 3304)	0.20725554375570943
  (1, 2521)	0.27298612616808393
  (1, 1901)	0.16126750217419497
  (1, 1799)	0.5459722523361679
  (1, 1461)	0.23288722901895947
  (1, 1174)	0.6009236411330827
  (1, 1115)	0.19055904297271162
  (2, 3426)	0.2806649461753161
  (2, 3294)	0.41045832620773526
  (2, 2504)	0.3939997819117998
  (2, 1901)	0.23275673961163454
  (2, 1363)	0.32656789329397123
  (2, 1267)	0.20927579874902472
  (2, 1115)	0.2750331030608531
  (2, 1089)	0.3619836706680804
  (2, 228)	0.34157798353563024
  :	:
  (357, 3316)	0.3363937138703973
  (357, 2922)	0.23648265018428938
  (357, 2550)	0.24697345118772907
  (357, 1852)	0.20572116820152855
  (357, 1838)	0.2957292465970144
  (357, 1407)	0.24410926792211493
  (357,

### Training The ML Model

### Logistic Regression

In [37]:
model = LogisticRegression(max_iter=1000)

In [38]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

### ML Model Evaluation

#### Accuracy Score

In [39]:
X_train_predictions = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_predictions)

In [40]:
print(f"Accuracy on the training data is {training_data_accuracy}")

Accuracy on the training data is 0.94375


In [41]:
X_test_predictions = model.predict(X_test)
testing_data_accuracy = accuracy_score(Y_test, X_test_predictions)

In [42]:
print(f"Accuracy on the testing data is {testing_data_accuracy}")


Accuracy on the testing data is 0.6944444444444444


#### Saving The Trained Model

In [43]:
import pickle

In [44]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [45]:
filename_vectorizer = 'vectorizer.sav'
pickle.dump(vectorizer, open(filename_vectorizer, 'wb'))

#### Using The Saved Model For Future Predictions

In [46]:
# loading the saved model
loaded_model = pickle.load(open('./trained_model.sav', 'rb'))

In [47]:
X_new = X_test[309]
print(Y_test[309])
prediction = model.predict(X_new)
print(prediction)
if (prediction[0] == 0):
    print("It is a negative tweet.")
    
else:
    print("It is a positive tweet.")

1
[1]
It is a positive tweet.
